In [15]:
#ordinalencoder
#labelencoder
#onehotencoder
#usage
#differences

In [38]:
import psycopg2 as pg2
#python-postgre interaction module
import pandas as pd
#dataframes and series
import re
#regex
import logins
conn = pg2.connect(database='dvd_rentals', user=logins.login,password=logins.master_pass)
cur = conn.cursor()
#connect to db and run sql commands to return data
cur.execute("""SELECT rental.rental_id,EXTRACT(DAY FROM return_date-rental_date),city.city,country.country,amount,rating,category.name
FROM rental
LEFT JOIN customer
ON customer.customer_id = rental.customer_id
LEFT JOIN payment
ON payment.rental_id = rental.rental_id
LEFT JOIN address
ON address.address_id = customer.address_id
LEFT JOIN city
ON city.city_id = address.city_id
LEFT JOIN country
ON country.country_id = city.country_id
LEFT JOIN inventory
ON inventory.inventory_id = rental.inventory_id
LEFT JOIN film
ON film.film_id = inventory.film_id
LEFT JOIN film_category
ON film_category.film_id = film.film_id
LEFT JOIN category
ON category.category_id = film_category.category_id""")
#store data locally
x = cur.fetchmany(5000)
#cur.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'film' ORDER BY ORDINAL_POSITION")
x1 = ['rental_id','days_rented','city','country','amount','rating','category']
#for i in cur.fetchall():
#    x1.append(i[0])
s = pd.Series(x1)
df1 = pd.DataFrame.from_records(x, columns=s.values)
df1 = df1.dropna(axis=0)
df1['days_rented'] = df1['days_rented'].apply(int)
df1 = df1.reset_index()
df1 = df1.drop(columns='index')
df1


,rental_id,days_rented,city,country,amount,rating,category
0,1520,8,Ede,Netherlands,7.99,R,Horror
1,1778,5,Ede,Netherlands,1.99,PG,Documentary
2,1849,6,Ede,Netherlands,7.99,PG,New
3,2829,4,Ede,Netherlands,2.99,G,Classics
4,3130,8,Ede,Netherlands,7.99,R,Games
...,...,...,...,...,...,...,...
4994,12693,7,Banjul,Gambia,5.99,PG,Sci-Fi
4995,12844,5,Banjul,Gambia,2.99,G,Animation
4996,14102,9,Banjul,Gambia,5.99,PG-13,Action
4997,14230,6,Banjul,Gambia,5.99,PG-13,Classics


In [39]:
cur.execute("SELECT * FROM customer")
cdata = cur.fetchall()
cur.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'customer' ORDER BY ORDINAL_POSITION")
ccolumns = list()
for i in cur.fetchall():
    ccolumns.append(i[0])
df = pd.DataFrame(data=cdata, columns=ccolumns)
df

,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active
0,524,1,Jared,Ely,jared.ely@sakilacustomer.org,530,True,2006-02-14,2013-05-26 14:49:45.738,1
1,1,1,Mary,Smith,mary.smith@sakilacustomer.org,5,True,2006-02-14,2013-05-26 14:49:45.738,1
2,2,1,Patricia,Johnson,patricia.johnson@sakilacustomer.org,6,True,2006-02-14,2013-05-26 14:49:45.738,1
3,3,1,Linda,Williams,linda.williams@sakilacustomer.org,7,True,2006-02-14,2013-05-26 14:49:45.738,1
4,4,2,Barbara,Jones,barbara.jones@sakilacustomer.org,8,True,2006-02-14,2013-05-26 14:49:45.738,1
...,...,...,...,...,...,...,...,...,...,...
594,595,1,Terrence,Gunderson,terrence.gunderson@sakilacustomer.org,601,True,2006-02-14,2013-05-26 14:49:45.738,1
595,596,1,Enrique,Forsythe,enrique.forsythe@sakilacustomer.org,602,True,2006-02-14,2013-05-26 14:49:45.738,1
596,597,1,Freddie,Duggan,freddie.duggan@sakilacustomer.org,603,True,2006-02-14,2013-05-26 14:49:45.738,1
597,598,1,Wade,Delvalle,wade.delvalle@sakilacustomer.org,604,True,2006-02-14,2013-05-26 14:49:45.738,1


In [18]:
#active,name,email

In [40]:
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import accuracy_score
import numpy as np
X = list()
y = list()
for i in range(len(df.index)):
    X.append([df.store_id[i],df.address_id[i]])
    y.append(df.active[i])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33)
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_pred,y_test)

0.9747474747474747

In [20]:
#category - country city,amount
#rating - country,city,amount
#return_date - country, city, amount

In [41]:
import numpy as np
from sklearn import datasets,preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
X = list()
y = list()
for i in range(len(df1.index)):
    X.append([df1.city[i],df1.country[i],df1.amount[i]])
    y.append((df1.category[i]))
enc = OneHotEncoder(handle_unknown = 'ignore')
X = enc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred,y_test)

0.071

In [42]:
import numpy as np
from sklearn import datasets,preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
X = list()
y = list()
for i in range(len(df1.index)):
    X.append([df1.city[i],df1.country[i],df1.amount[i]])
    y.append((df1.rating[i]))
enc = OneHotEncoder(handle_unknown = 'ignore')
X = enc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred,y_test)

0.187

In [46]:
import numpy as np
from sklearn import datasets,preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
X = list()
y = list()
for i in range(len(df1.index)):
    X.append([df1.city[i],df1.country[i],df1.amount[i]])
    y.append((df1.days_rented[i]))
enc = OneHotEncoder(handle_unknown = 'ignore')
X = enc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred,y_test)


0.21

In [ ]:
#find best model
#save best model (pickles)
#run test data through pickles and score
#transform from df to numpy array and run ml model with numpy
#evaluate the model executed from numpy input
#compare grid search and random search
#differences label/onehot/ordinal encoder, what and when